# Pickhardt Payments Simulator

This is an educational simulation to demonstrate how to implement the main concepts of [probabilistic payment delivery with the additional maintainance of the uncertainty network](https://arxiv.org/abs/2103.08576) and [optimal splitting of payments in MPP which is also known as optimally realiable payment flows](https://arxiv.org/abs/2107.05322)

This code assumes an oracle with the actual liquidity of channels to exist. In reality the oracle is the lightning network which can be queried by sending out actual onions. We randomly generate the liquidity for the oracle but one may also obviously use a crawl of the network


## Warning: THIS IS WORK IN PROGRESS: The code has some severe limitations...
... which may make it impractical for real nodes and implementations. Do not just blindly wrap this into production code but address the following limitations:

* Software tests are missing
* channel reserves are ignored
* HTLC limits like min_size, max_size, max_accepted_htlc are fully ignored
* strategies to operate with hanging htlc's (some might require protocol updates) may need to be included
* concurrent handeling of onions and maintanance of the uncertainty network
* proper handling of non zero base fee channels is needed and in particular handeling their cost properly at least as good as it is possible
* better piecewise splitting / quantization of the amounts is necessary
* learning how to weight various features needs to be conducted or at least configureably be in place or...
* ...proper feature Engineering. Combining the linearized integer uncertainty cost and the routing cost just via the weight $\mu$ seens a bit arbitrary. I think centering, scaling and especially for channels log transformation etc as [described in this article](https://www.kaggle.com/code/milankalkenings/comprehensive-tutorial-feature-engineering/notebook) might work better
* Mechanics of making learnt information persistant over some time
* inclusion of other optimization goals that predict reliability are missing (like latency, distance, ...)
* One should Prune the graph before invoking the solver (I tested but excluded a totally arbitrary pruning mechanism which produced a speedup of 10x for the solver making it run constantly in less than 100ms). I am very sure with centering, scaling and proper feature engineering we can get a much better pruner with errror guarantees and similar or even better speedup)
* properly conduct the piecewise linearization of the cost function
* don't overwrite memory of the entire UncertaintyNetwork and UncertaintyChannel and the solver all the time but just update the necessary pieces before invoking the solver
* potentially write your own stand alone cost scaling min cost flow solver instead of relying on a third party dependency
* Onions of upstream channels do not add downstream fees which would have to be done in mainnet
* **IMPORTANT** Proper error handling of the ortools lib if not enough liquidity is on the network and no flow is feasible
 
## What the code does: 
* Conduct and simulate a single PaymentSession 
* Properly track the learnt information from prior Attempts
* Showcase how strategies similar to the BOTL 14 proposal may be used
* Show how a sequential pay loop may conceptionally be built
* Properly transform the payment delivery problem to a min cost flow problem with a piecewise linearized cost function
* Demonstrate that the min cost flow approach is feasbale. 
* Allow for extension 
* proper handling of parallel channels! (YES! I finally handle them properly. I always virtualized and simplified but I came around)


## BOLT14 considerations

This code also contains a few experiments to investigate the effects of the BOLT14 proposal.
Maybe it might make sense to make entropy hints not only as a foaf query but as an n-bit network wide gossip message. Unless we have some prior belief about the channel the 2 bit gossip intervalls are

* [0, c/4]
* [c/4,c/2]
* [c/2,3c/4]
* [3c/4,c]

nodes would only signal a message if the liquidity switches from one of those intervals to another (of course one could also require 3 bits or so)

## TODOS: 

* Make it easier to use with other implementations than c-lighting by creating a mapping from your channel data to cln_json and use 
* make bolt14 experiemtns more realistic and depend on actual `send_onions` calls
* as hinted in the ldk issue https://github.com/lightningdevkit/rust-lightning/issues/1372 it might make sense to not use a global \mu but rather learn it for each channel by comparing min uncertainty cost ~ 1/c max uncertainty cost = log(c+1) with the ppm on the edge and make sure we get this in the same order. It is only a test but it might turn out to work really well
* fix some fixme's most notably the data model that produces poor probability computation in outputs as it ignors the prior knowlege which means all displayed probabilities are too low (though the flow is computed properly)
* make a useful design for liquidity sharing and BOLT 14 experiments
* Extract standalone python module with Channel, UncertaintyChannel, ChannelGraph & UncertaintyNetwork. Also provide clearer Interface for people to implement their own Oracles
* provide a few more examples / simulations and in particular derrive some nice diagrams sumarizing some results

# Funding

This research result is funded through [NTNU](https://www.ntnu.no) and [BitMEX](https://blog.bitmex.com/bitmex-2021-open-source-developer-grants/) as well as generous donations from the Bitcoin community via https://donate.ln.rene-pickhardt.de and via recurring donations at https://www.patreon.com/renepickhardt. If you want to learn why independent research and development for Bitcoin and the Lightning Network is important I kindly refer you to: https://ln.rene-pickhardt.de/ of course I will be grateful if you consider my work of importance and decide to support it.

Special Thanks to Stefan Richter with whom we implemented the first mainnet tests on top of lnd which certainly helped me to clarify many details and get the maintanance of the uncertainty network straight. Thanks to Michael Ziegler and Carsten Otto for an initial code review that made me rewrite the entire code base from scratch.

# Glossary of terms

As many similar terms are being used for similar concepts and things and since I have been studying these topics for three years now I will follow the good practice of [BOLT0](https://github.com/lightning/bolts/blob/master/00-introduction.md) and put a glossary of used terms up here with the hope that I finally have some good, useful and precise defintions that will also bee picked up by others.


## (Payment) Channel
Payment `channel`s are either announced or unnannounced but known to the sender and recpient of a payment. They have meta data like the `capacity` or `routing fees` as well as some other config values which we carry around but ignore for simplicity here.

The `Channel` is the base class in this code and identifies a channel by the trippled `(source_node_id, destionation_node_id, short_channel_id)` in this way we make the direction explicit and don't encode about lexicographical DER-encoding of `node_ids` and the direction field as done by the bolts.

##  Arc
An `arc` is not to be confused with a payment `channel`. `Arc`s are the data structure that arise from the piece wise linearized `cost function` and are used internally in the min cost flow solver. In this code you will not find the terminology of an arc.

## edge
eges are the things between nodes of a graph or network. In the code this is just the native name of the networkx library. Edges usually map to `Channels`, `UncertaintyChannels` or `OracleChannels` and usually have one of those attached to it in the `channel`-field.


## (Channel) Capacity
The `capacity` of a payment channel is its size as observed in the amount of the output of the funding transaction which is announced via the `short_channel_id` in gossip.

**Important**: It seems to be an unavoidable collision that the term capacity is also used for `arcs` in min cost flow solvers. As I introduce the piece wise linearization the `cost function` the arcs in the min cost flow solve will usually have a lower `capacity` than the cannels they represent. I try to distinguish this by talking either about the `channel capacity` or the `arc capacity` if absolutely necessary.

## Liquidity
The `Liquidity` is the amount of satoshis a peer has available to route on it's channel. This means that unless the (even) `capacity` is split exactly 50/50 between the peers the `liquidity` in a channel will be different for both peers. 

typical terms in the wild: balance, local balance, outbound capacity

## Uncertainty / Entropy 
assuming a prior probability distribution we can quantify the `uncertainty` about the channels `liquidity` by measureing the `entropy` of the probability function. Notably if a channel as a `capacity` of `c` satoshis the entropy is `log(c+1)`

## Unertainty Network

The `uncertainty network` is a model than encodes our current belief about the available `liquidity` in the channels of the Lightning network. This is usually achieved by storing three values: 

1. Minimum `liquidity` which initialy for remote channel is `0`. 
2. Maximum `liquidity` which initially for remote channels corresponds its `capacity`.
3. `inflight` is the number of satoshis that we currently have allocated to this channel in outstanding onions and HTLCs.

## inflight
We use `inflight` to encode how many satoshis we have currently allocated or plan to allocate to a channel. Note that this is not the same as the number of inflight htlcs that the peer co-owning the channel will observe as some of the onions that we have outstanding may not have been delivered to the channel yet or may have failed but not returend back to us yet.

## Cost(function)

The `cost` function encodes our optimization goals and consists of several `features`. typical `features` are the `uncertainty cost` and the `routing fees`. Note that the same cost function can be applied to Dijkstra's algorithm as to min cost flow solvers. However in the general case of min cost flows the cost function should be convex or even better linear or piecewise linear so that a reasonable runtime can be achieved. 

The `cost` is just an internal value that will be minimized in computation of `candidate paths` for the `payment loop`. This cost may or may not translate to `routing fees`.

typical terms in the wild: Score, fees, penalty

## Unit cost
Linear min cost flow solvers need a fixed cost for transporting 1 unit (e.g. 1 satoshi) along an `arc` (e.g. `channel`). Oft

## Uncertainty cost
This is the `feature` introduced in the [`probablistic payment delivery` paper](https://arxiv.org/abs/2103.08576) it puts a cost on an amount `a` to be send over a (`path`) of `payment channels`.
It is comuted by taking the negative logarithm of the `success probability` if we have no prior belief of the channel the `uncertainty cost` will be computed as: `-log((c+1-a)/(c+1))`. Note that the uncertainty cost is a convex gowing function takin the value `0 = -log((c+1-0)/(c+1)) = -log(1)` if no sats are to be allocated on the channel. The maximum possible `uncertainty cost` occurs if `a=c` and takes the value `log(c+1)` which is exactly the `entropy` of the channel which can be seen by the following calculation:
`-log((c+1-c)/(c+1)) = -log(1/(c+1)) = log(c+1)`

### linearized uncertainty cost
The linearized uncertainty cost is the first term of the taylor approximation of `f(a)=-log((c+1-a)/(c+1))` which results in `a/(c+1)`. This (in my opinion surprisingly) corresponds to the failure probability. Note that we should not interpret it as such as it does not make sense to add probabilities while those feature will be added in the global `cost function` of the min cost flow solvers. 

This results in a `unit` cost of `1/(c+1)`. since integers are preferable we multiply the unit cost with the maximum observed channel `capacity` of all channels and ignoring the decimals. 

Thus the **final integer linearized uncertainty unit cost** is: `int(c_{max}/(c+1))`

## (routing) fees

The routing `fee` is a feature that usually corresponds directly to the fees that are being charged to forward a payment of size `a` along a channel. According to the protocol it is currently being computed as: 

`fee(a) = a*ppm/1000 + base_fee_msat`

## linearized fees

As discussed in our research and on the mailinglist the fee function of the lightning network is not linear. In the mentioned discussion one can see that `fee(a+b) = fee(a)+fee(b) - base_fee_msat` which is linear if and only if `base_fee_msat = 0`. 

Thus in computation we currently linearize the `fee` function by ignoring the `base fee` so we use `fee(a) = a*ppm/1000` giving us a `unit cost` of `ppm/1000` 

As min cost flow solvers prefer integer cost we scale by `1000` and arrive at the **final integer linearized fee unit cost** which is just the: `ppm`. 

For this to work good enough we might also take channel that charge a very small `base_fee` and later just pay it when we send the onions.


## Features

The `features` of our cost function encode our optimization goals. Typicalfeatures goals might be to have a cheap price (usually encoded by `fee`) or to have a high reliability (in this work best encoded by the `uncertainty cost`that comes from the estimated `success probability` that the channel has enough `liquidity`. Other features that seam reasonable are things related to `latency` of payments (e.g. [pyhsical geodistance or virtaul IP-distance as indicted in this comment](https://github.com/lightningdevkit/rust-lightning/issues/1170)). Implementations have also experimented with other features like CLTV delta or channel age. 

Feature engineering [as described in this article](https://www.kaggle.com/code/milankalkenings/comprehensive-tutorial-feature-engineering/notebook) is a crucial part for future research 

## Success Probability

The definition of the `success probability` is in our case modelled by estimating the likelihood that a channel with a `capacity` of `c` has `a` sats of `liquidity` available. 

In the base case where we have not learnt anything about the channel's `liquidity` in the past the `success probability` for making a payment of size `a` stays at `P(X>=a) = (c+1-a)/(c+1)`

Let's say a payment delivery algorithm sends an onion via a path of channel and wants to add an HTLC with amount `h` to a channel of capacity `c` there are several cases that we can observe to update our uncertainty about the network for future payments of size `a`

1. Payment failed at the channel: This means the channel of capacity `c` has at most `h-1` satoshi in it. In Terms of Probability theory we can ask ourselves what is the success rate for a subsequent payment of size `a` given the event `X<h` (the htlc  `h` has failed) which can be expressed as `P(X>=a | X<h)`. Here we see because of the condition that for `a>=h` the proabbility has to be `0`. Computing the conditional probability for the remainder in the uniform case we can se `P(X>=a | X < h) = (h-a)/h`. Note that this is the same as the success probability for a channel of capacity `h-1`. 
2. Payment failed at a downstream channel: This means that the channel of capcity `c` was able to route `h` satoshi and has at least a minimum `liquidity` of `h`. This means that `P(X>=a | X>=h) = 1`  for all amounts `a <=h` and in the unform case for all larger values of `a` the conditional probability materializes to: `P(X>=a)/P(X>=h) = ((c+1-a)/(c+1))/((c+1-h)/(c+1)) = (c+1-a)/(c+1-h)` Note that this fraction is always smaller than `1` as `a >=h`
3. The case that the payment was successfull or did not return an error: Following the same thoughts and the first bullet point at the end of section 3 in the second paper we know that for a future payment of size `a` we have to look at `P(X>=a + h | X >=h)` which in the uniform case materializes to `( (c-h) + 1 - a)/( (c - h) + 1)`. Note that this is the same as if the channel shrunk from `c` to `c-h` as we now know that the maximum `liquidity` is not `c` but rather `c-h`


## optimal payment flow / optimal mpp split

We call a payment flow optimal if it minimizes the `cost function` that encodes our optimization goals. Note that the solution of the min cost flow problem also defines a split of the payment across various paths. While simple algorithms for discecting a flow into paths exist the actual disection is not uique and needs further research. Especially when we start taking channel configurations into account.

## Payment Loop

The payment loop is the trial and error process of delivering a certain amount of satoshis from a sending node to a recipient. This is usually done by generating a set of candidate onions from the solution of an `optimal payment flow`. The onions are concurrently being sent out and error are being used to update our belief about the `liquidity` in the `uncertainty network`.

## Probabilistic Payment Delivery

The idea of `probabilistic payment delivery` is to send onions that maximize the success proabbility for all (or parts of them) to be successul. This takes the `uncertainty` about the remote `liquidity` into consideration and was first introduced in path finding (Note I did not use the term path findng nore did I put it in the glossary)

## Pickhardt Payments
As the term is more and more starting to flow around I will try to summarize best what is currently from a technial point of view meant by it: 

`Using probabilistic payment delivery` in a round based `payment loop` that updeates our belief of the remote `liquidity` in the `uncertainty network` and generates reliable and cheap `payment flows` in every round by solving a `piece wise linearized min integer cost flow problem with a seperable cost function` (I start to see why a shorter term was needed).



# Imports from the Python standard lib

We could get rid of all these dependencies! 
* `json` is only used to parse and store the information from `lightning-cli listchannels`. 
* `random` is used to feed our oracle with a simulated ground truth. If you make actual payments the mainnet Lightnign Network will act as an oracle
* `time` is only used for time measurements
* While `log` is used to measure `entropy` and `uncertainty cost` the linerized problem gets rid of the `log` and the Entropy is also only measured for experimental data.
* the `typing` lib is just for type safty

In [1]:
import json
import random
import time
from math import log2 as log
from typing import List


# Imports of third party libraries

We use `networkx` to internally store the `ChannelGraph`, the `UncertaintyNetwork` and the `OracleLightningNetwork` this produces quite some inefficient memory management an decreases the runtime of this simulation significantly. Yet for the sake of simplicity and readability of the code and to explain the concepts we chose to use the `networkx` for this task.

For solving the min cost flow problem we used `ortools` from the Google OR-Tools lib. This is a highly efficient C++ library that contains an integer linear min cost flow solver. We loose most of our runtime by maintaining the the state of our model and feeding it into the min cost flow solver provided by `ortools`. If you want to create an actual pay implementation we suggest that you extract the solver from `ortools` and maintain the state of the model in a way that is memory efficient.

In [2]:
import networkx as nx
from ortools.graph import pywrapgraph

# Some config values

In [3]:
#ignores all channels with a base fee higher than DEFAULT_BASE_THRESHOLD msat
DEFAULT_BASE_THRESHOLD = 0

#mu value to combine both features
DEFAULT_MU = 1

#default number of piecewise linear approximations of the uncertainty cost
DEFAULT_N = 5

## Channel

The `Channel` is used to store publicly available infromation about a channel. It currently follows mainly the format of `lightning-cli listchannels` output from `c-lightning`. The class is intended to only store the gossip information and not meant to modify the information. In a mainnet `pickhardt-pay` implementation one would obviously want a class that also processes incoming `update_channel` messages.

`Channels` are identified by the trippled `(source_node_id, destionation_node_id, short_channel_id)` in this way we make the direction explicit and don't encode about lexicographical DER-encoding of `node_ids` and the direction field as done by the bolts.


In [4]:
class ChannelFields():
    """
    These are the values describing public data about channels that is either available
    via gossip or via the Bitcoin Blockchain. Their format is taken from the c-lighting
    API. If you use a different implementation I suggest to write a wrapper around the
    `ChannelFields` and `Channel` class
    """
    SRC = 'source'
    HTLC_MINIMUM_MSAT = 'htlc_minimum_msat'
    HTLC_MAXIMUM_MSAT = 'htlc_maximum_msat'
    BASE_FEE_MSAT = 'base_fee_millisatoshi'
    ANNOUNCED = 'public'
    DEST = 'destination'
    LAST_UPDATE = 'last_update'
    FEE_RATE = 'fee_per_millionth'
    FEATURES = 'features' 
    CAP = 'satoshis'
    ACTIVE = 'active'
    CLTV = 'delay'
    FLAGS = 'channel_flags'
    SHORT_CHANNEL_ID = 'short_channel_id'

class Channel():
    """
    Stores the public available information of a channel.
    
    The `Channel` Class is intended to be read only and internatlly stores
    the data from c-lightning's `lightning-cli listchannels` command as a json.
    If you retrieve data from a different implementation I suggest to overload
    the constructor and transform the information into the given json format
    """
    def __init__(self,cln_jsn):
        self._cln_jsn = cln_jsn
        
    @property
    def cln_jsn(self):
        return self._cln_jsn
    
    @property
    def src(self):
        return self._cln_jsn[ChannelFields.SRC]

    @property
    def htlc_min_msat(self):
        return self._cln_jsn[ChannelFields.HTLC_MINIMUM_MSAT]

    @property
    def htlc_max_msat(self):
        return self._cln_jsn[ChannelFields.HTLC_MAXIMUM_MSAT]

    @property
    def base_fee(self):
        return self._cln_jsn[ChannelFields.BASE_FEE_MSAT]

    @property
    def is_announced(self):
        return self._cln_jsn[ChannelFields.ANNOUNCED]

    @property
    def dest(self):
        return self._cln_jsn[ChannelFields.DEST]

    @property
    def ppm(self):
        return self._cln_jsn[ChannelFields.FEE_RATE]

    @property
    def capacity(self):
        return self._cln_jsn[ChannelFields.CAP]

    @property
    def is_active(self):
        return self._cln_jsn[ChannelFields.ACTIVE]

    @property
    def cltv_delta(self):
        return self._cln_jsn[ChannelFields.CLTV]
    
    @property
    def flags(self):
        return self._cln_jsn[ChannelFields.FLAGS]

    @property
    def short_channel_id(self):
        return self._cln_jsn[ChannelFields.SHORT_CHANNEL_ID]
    
    def __str__(self):
        return str(self._cln_jsn)



# ChannelGraph

The `ChannelGraph` is a term borrowed from `lnd`. While we also have the `UncertaintyNetwork` and the `OracleLightningNetwork` we decided to take over the `lnd` notation because it is wildly used

The `ChannelGraph` is the most basic data structure and extended to the `UncertaintyNetwork` and `OracleNetwork`. Note that we didn't use the Term `Network` or `ChannelNetwork` which from a software engineering perspecive would have been more resonable. There was [a poll deciding against](https://twitter.com/renepickhardt/status/1513095719862816769) this.

In [5]:
class ChannelGraph():
    """
    Represents the public information about the Lightning Network that we see from Gossip and the 
    Bitcoin Blockchain. 
    
    The channels of the Channel Graph are directed and identiried uniquly by a triple consisting of
    (source_node_id, destination_node_id, short_channel_id). This allows the ChannelGraph to also 
    contain parallel channels.
    """
    def _get_channel_json(self,filename:str):
        """
        extracts the dictionary from the file that contains lightnig-cli listchannels json string
        """
        f = open(filename)
        return json.load(f)["channels"]
    
    
    def __init__(self,lightning_cli_listchannels_json_file:str):
        """
        Importing the channel_graph from c-lightning listchannels command the file can be received by 
        #$ lightning-cli listchannels > listchannels.json

        """
        
        self._channel_graph = nx.MultiDiGraph()
        channels = self._get_channel_json(lightning_cli_listchannels_json_file)
        for channel in channels:
            channel = Channel(channel)
            self._channel_graph.add_edge(channel.src, channel.dest, key=channel.short_channel_id, channel=channel)
            
    @property
    def network(self):
        return self._channel_graph
    
    
    def get_channel(self, src: str, dest: str, short_channel_id: str):
        """
        returns a specific channel object identified by source, destination and short_channel_id
        from the ChannelGraph
        """
        if self.network.has_edge(src,dest):
            if short_channel_id in self.network[src][dest]:
                return self.network[src][dest][short_channel_id]["channel"]


# Oracle Channel

While on the Lightning Network we cannot ask a channel for its actual liquidity we decided for the simplicity of the simulation to not only have the `can_forward` API call but also provide `actual_liquidity` API. This makes some data flow easier. Of course this information must not be used to feed the min cost flow solver in the actual experiments

In [6]:
class OracleChannel(Channel):
    """
    An OracleChannel us used in experiments and Simulations to form the (Oracle)LightningNetwork.
    
    It contains a ground truth about the Liquidity of a channel
    """    
    def __init__(self, channel:Channel, actual_liquidity : int = None):
        super().__init__(channel.cln_jsn)
        self._actual_liquidity=actual_liquidity
        if actual_liquidity is None or actual_liquidity >= self.capacity or actual_liquidity < 0:
            self._actual_liquidity = random.randint(0,self.capacity)
    
    def __str__(self):
        return super().__str__()+" actual Liquidity: {}".format(self.actual_liquidity)
    
    @property
    def actual_liquidity(self):
        """
        Tells us the actual liquidity according to the oracle. 
        
        This is usful for experiments but must of course not be used in routing and is also
        not a vailable if mainnet remote channels are being used.
        """
        return self._actual_liquidity

    def can_forward(self,amt:int):
        """
        check if the oracle channel can forward a certain amount
        """
        if amt <= self.actual_liquidity:
            return True
        else:
            return False    

# Oracle Lightning Network

This is a very simplified class that can easily be used in simulations. It randomly assigns liquidity to channels of an existing ChannelGraph following a uniform prior. 

One might want to have interfaces to import liquidity information from crawls of the network or instead or use the live mainnet Lightning Network as an oracle

In [7]:

class OracleLightningNetwork(ChannelGraph):
            
    def __init__(self,channel_graph:ChannelGraph):
        self._channel_graph = channel_graph
        self._network = nx.MultiDiGraph()
        for src,dest, short_channel_id, channel in channel_graph.network.edges(data="channel", keys=True):
            oracle_channel = None
            
            #If Channel in oposite direction already exists with liquidity information match the channel
            if self._network.has_edge(dest,src):
                if short_channel_id in self._network[dest][src]:
                    capacity = channel.capacity
                    opposite_channel = self._network[dest][src][short_channel_id]["channel"]
                    opposite_liquidity = opposite_channel.actual_liquidity
                    oracle_channel = OracleChannel(channel, capacity - opposite_liquidity)
                    
            
            if oracle_channel is None:
                oracle_channel = OracleChannel(channel)
            
            self._network.add_edge(oracle_channel.src, 
                                          oracle_channel.dest, 
                                          key=short_channel_id, 
                                          channel=oracle_channel)
        
    @property
    def network(self):
        return self._network
            
    def send_onion(self,path,amt):
        for channel in path:
            oracle_channel = self.get_channel(channel.src, channel.dest, channel.short_channel_id)
            success_of_probe = oracle_channel.can_forward(channel.in_flight+amt)
            #print(channel,amt,success_of_probe)
            channel.update_knowledge(amt,success_of_probe)
            if success_of_probe == False:
                return False, channel
        return True, None
    
    def theoretical_maximum_payable_amount(self, source : str, destination : str, base_fee : int = DEFAULT_BASE_THRESHOLD):
        """
        Uses the information from the oracle to compute the min-cut between source and destination
        
        This is only useful for experiments and simulations if one wants to know what would be 
        possible to actually send before starting the payment loop
        """        
        test_network = nx.DiGraph()
        for src,dest,channel in self.network.edges(data="channel"):
            #liqudity = 0
            #for channel in channels:
            if channel.base_fee > base_fee:
                continue
            liquidity = self.get_channel(src,dest,channel.short_channel_id).actual_liquidity
            if liquidity > 0:
                if test_network.has_edge(src,dest):
                    test_network[src][dest]["capacity"]+=liquidity
                else:
                    test_network.add_edge(src,
                                      dest,
                                      capacity = liquidity)
        
        mincut, _ = nx.minimum_cut(test_network, source, destination)
        return mincut

# Uncertainty Channel Class

The uncertainty channel is the core object for the uncertainty network and is where all the magic happens. Some API calls have been included even though we don't need them to precisely explain the theory and where everything is coming from. The reasoning for all the various costs and how to derive at the linearized integer unit costs is in the glossary. 

In [8]:
class UncertaintyChannel(Channel):
    """
    The channel class contains basic information of a channel that will be used to create the
    UncertaintyNetwork.
    
    Since we optimize for reliability via a probability estimate for liquidity that is based
    on the capacity of the channel the class contains the `capacity` as seen in the funding tx output.
    
    As we also optimize for fees and want to be able to compute the fees of a flow the classe
    contains information for the feerate (`ppm`) and the base_fee (`base`). 
    
    Most importantly the class stores our belief about the liquidity information of a channel.
    This is done by reducing the uncertainty interval from [0,`capacity`] to 
    [`min_liquidity`, `max_liquidity`].
    Additionally we need to know how many sats we currently have allocated via outstanding onions
    to the channel which is stored in `inflight`.
    
    The most important API call is the `get_piecewise_linearized_costs` function that computes the
    pieceweise linearized cost for a channel rising from uncertainty as well as routing fees.
    """
    
    TOTAL_NUMBER_OF_SATS = 21_000_000 * 100_000_000
    MAX_CHANNEL_SIZE = 15_000_000_000 #150 BTC

    def __init__(self,cln_jsn):
        super().__init__(cln_jsn)
        self.forget_information()
    
    def __init__(self, channel:Channel):
        super().__init__(channel.cln_jsn)
        self.forget_information()
    
    def __str__(self):
        return "Size: {} with {:4.2f} bits of Entropy. Uncertainty Interval: [{},{}] inflight: {}".format(
            self.capacity, 
            self.entropy(), 
            self.min_liquidity, 
            self.max_liquidity, 
            self.in_flight)

    @property
    def max_liquidity(self):
        return self._max_liquidity

    @property
    def min_liquidity(self):
        return self._min_liquidity

    @property
    def in_flight(self):
        return self._in_flight

    #FIXME: store timestamps when using setters so that we know when we learnt our belief
    @min_liquidity.setter
    def min_liquidity(self,value:int):
        self._min_liquidity=value

    #FIXME: store timestamps when using setters so that we know when we learnt our belief
    @max_liquidity.setter
    def max_liquidity(self,value:int):
        self._max_liquidity=value

    #FIXME: store timestamps when using setters so that we know when we learnt our belief
    @in_flight.setter    
    def in_flight(self,value:int):
        self._in_flight=value
        
    @property
    def conditional_capacity(self,respect_inflight=True):
        #FIXME: make sure if respect_inflight=True is needed for linearized cost
        if respect_inflight == False:
            return self.max_liquidity - self.min_liquidity
        
        min_liquidity = max(self.min_liquidity,self.in_flight)
        return max(self.max_liquidity - min_liquidity,0)

        
    def allocate_amount(self,amt:int):
        """
        assign or remove ammount that is assigned to be `in_flight`.
        """
        self.in_flight += amt
        if self.in_flight < 0:
            raise Exception("Can't remove in flight HTLC of amt {} current inflight: {}".format(-amt,self._in_flight-amt))

    #FIXME: store timestamps when using setters so that we know when we learnt our belief
    def forget_information(self):
        """
        resets the information that we belief to have about the channel. 
        """
        self.min_liquidity=0
        self.max_liquidity=self.capacity
        #FIXME: Is there a case where we want to keep inflight information but reset information?
        self.in_flight = 0

    
    def entropy(self):
        """
        returns the uncertainty that we have about the channel

        this respects our belief about the channel's liquidity and thus is just the log
        of the conditional capacity.

        FIXME: This respects inflight information? I assume it shouldn't.
        """
        return log(self.conditional_capacity + 1)
    
    def success_probability(self,amt:int = None):
        """
        returns the estimated success probability for a payment based on our belief about the channel using a uniform distribution.
        
        While this is the core of the theory it is only used for evaluation and not for the
        actual min cost flow computation as we linearize this to an integer unit cost
        
        In particular the conditional probability P(X>=a | min_liquidity < X < max_liquidity)
        is computed based on our belief also respecting how many satoshis we have currently 
        outstanding and allocated. Thus it is possible that testing for the `amt=0` that the success probability
        is zero and in particular not `1`.
        
        It also accounts for the number of satoshis we have already outstanding but have not received information about
        
        FIXME: Potentially test other prior distributions like mixedmodels where most funds are on one side of the channel
        """
        if amt is None:
            amt = 0
        tested_liquidity = amt + self.in_flight
        if tested_liquidity <= self.min_liquidity:
            return 1.0
        elif tested_liquidity >= self.max_liquidity:
            return 0.
        else:            
            conditional_amount = tested_liquidity - self.min_liquidity
            #TODO: can't use self.condition_capacity as that respects inflight htlcs
            conditional_capacity= self.max_liquidity - self.min_liquidity
            if conditional_amount > conditional_capacity:
                return 0.
            return float(conditional_capacity + 1 - conditional_amount)/ (conditional_capacity + 1)
      
    def uncertainty_cost(self,amt:int):
        """
        Returns the uncertainty cost associated to sending the amount `amt` respecting our current belief
        about the channel's liquidity and the in_flight HTLC's that we have allocated and outstanding.
        """
        return -log(self.success_probability(amt))

    def linearized_uncertainty_cost(self,amt:int):
        """
        the linearized uncertainty cost is just amt/(capacity+1). Using this is most likely not what
        one wants as this tends to saturate channels. The API is included to explain the theory.
        
        Warning: This API does not respect our belief about the channels liquidity or allocated in_flight HTLCs
        """
        #TODO: Maybe change to `return amt*self.linlinearized_integer_uncertainty_unit_cost()`
        return float(amt)/(self.capacity+1)
    
    def linearized_integer_uncertainty_unit_cost(self,use_conditional_capacity=True):
        """
        estimates the linearized integer uncertainty cost
        
        FIXME: Instead of using the maximum capacity on the network it just assumes 150BTC to be max
        """
        #FIXME: interesting! Quantization does not change unit cost as it cancles itself
        #FIXME: use max satoshis available and control for quantization (makes mu depend on quantization....)
        if use_conditional_capacity:
            #FIXME: better choice of magic number but TOTAL_NUMBER_OF_SATS breaks solver
            return int(self.MAX_CHANNEL_SIZE/self.conditional_capacity)
            #return int(self.TOTAL_NUMBER_OF_SATS/self.capacity)
        else:
            return int(self.MAX_CHANNEL_SIZE/self.capacity)
            #return int(self.TOTAL_NUMBER_OF_SATS/self.conditional_capacity)
    
    
    def routing_cost_msat(self,amt:int):
        """
        Routing cost a routing node will earn to forward a payment along this channel in msats
        """
        return int(self.ppm*amt/1000) + self.base_fee
    
    def linearized_routing_cost_msat(self,amt:int):
        """
        Linearizing the routing cost by ignoring the base fee.
        
        Note that one can still include channels with small base fees to the computation the base 
        will just be excluded in the computation and has to be paid later anyway. If as developers
        we go down this road this will allow routing node operators to game us with the base fee
        thus it seems reasonable in routing computations to just ignore channels that charge a base fee.
        
        There are other ways of achieving this by overestimating the fee as ZmnSCPxj suggested at:
        https://lists.linuxfoundation.org/pipermail/lightning-dev/2021-August/003206.html
        """
        return int(self.ppm*amt/1000.)
    
    def linearized_integer_routing_unit_cost(self):
        "Note that the ppm is natively an integer and can just be taken as a unit cost for the solver"
        return int(self.ppm)
    
    def combined_linearized_unit_cost(self,mu :int =DEFAULT_MU):#FIXME: better default mu
        """
        Builds the weighted sum between our two unit costs.
        
        Not being used in the code. Just here to describe the theory.
        """
        return self.linearized_integer_uncertainty_unit_cost() + mu * self.linearized_integer_routing_unit_cost()
    
    def get_piecewise_linearized_costs(self,number_of_pieces : int = DEFAULT_N,
                                       mu : int = DEFAULT_MU):
        """
        
        """
        #FIXME: compute smarter linearization eg: http://www.iaeng.org/publication/WCECS2008/WCECS2008_pp1191-1194.pdf
        pieces = []*number_of_pieces

        #using certainly available liquidity costs us nothing but fees
        if int(self.min_liquidity-self.in_flight) > 0:
            uncertintay_unit_cost = 0 #is zero as we have no uncertainty in this case!
            pieces.append((int(self.min_liquidity-self.in_flight),uncertintay_unit_cost + mu * self.linearized_integer_routing_unit_cost()))
            number_of_pieces-=1

        # FIXME: include the in_flight stuff
        if int(self.conditional_capacity) > 0 and number_of_pieces > 0:
            arc_capacity = int(self.conditional_capacity/number_of_pieces)
            uncertintay_unit_cost = self.linearized_integer_uncertainty_unit_cost()
            for i in range(number_of_pieces):
                pieces.append((arc_capacity,(i+1)*uncertintay_unit_cost + mu * self.linearized_integer_routing_unit_cost()))
        return pieces
    
    """
    #FIXME: interestingly the following feature engineering does not work at all
    
    TODO: Look at more standard Univariate Transformations on Numerical Data techniques as described at 
    https://www.kaggle.com/code/milankalkenings/comprehensive-tutorial-feature-engineering/notebook
    
    def get_piecewise_linearized_costs(self,number_of_pieces : int = DEFAULT_N,
                                       mu : int = DEFAULT_MU,
                                       quantization : int = DEFAULT_QUANTIZATION):
        #FIXME: compute smarter linearization eg: http://www.iaeng.org/publication/WCECS2008/WCECS2008_pp1191-1194.pdf
        pieces = []*number_of_pieces

        #using certainly available liquidity costs us nothing but fees
        if int((self.min_liquidity-self.in_flight)/quantization) > 0:
            uncertintay_unit_cost = 0 #is zero as we have no uncertainty in this case!
            pieces.append((int((self.min_liquidity-self.in_flight)/quantization),uncertintay_unit_cost + mu * self.linearized_integer_routing_unit_cost()))
            number_of_pieces-=1

        # FIXME: include the in_flight stuff
        if int(self.conditional_capacity/quantization) > 0 and number_of_pieces > 0:
            capacity = int(self.conditional_capacity/(number_of_pieces*quantization))
            uncertintay_unit_cost = self.linearized_integer_uncertainty_unit_cost()
            for i in range(number_of_pieces):
                a = (i+1)*uncertintay_unit_cost+1
                b = self.linearized_integer_routing_unit_cost()+1
                pieces.append((capacity, int(a*b/(a+mu*b)) ))
        return pieces
    """

    def update_knowledge(self,amt:int,success_of_probe):
        """
        updates our knowledge about the channel if we tried to probe it for amount `amt`
        
        This API works ony if we have an Oracle that allows to ask the actual liquidity of a channel
        In mainnet Lightning our oracle will not work on a per_channel level. This will change the data
        flow. Here for simplicity of the simulation we make use of the Oracle on a per channel level
        """
        if success_of_probe:
            self.min_liquidity = max(self.min_liquidity,self.in_flight+amt)
        else:
            self.max_liquidity = min(self.max_liquidity, self.in_flight+amt)
    
    #needed for BOLT14 test experiment
    def learn_n_bits(self,oracle: OracleLightningNetwork, n : int = 1):
        """
        conducts n probes of channel via binary search starting from our belief
        
        This of course only learns `n` bits if we use a uniform success probability as a prior
        thus this method will not work if a different prior success probability is assumed 
        """
        if n <= 0:
            return
        amt = self.min_liquidity + int((self.max_liquidity - self.min_liquidity)/2)
        oracle_channel = oracle.get_channel(self.src, self.dest, self.short_channel_id)
        success_of_probing = oracle_channel.can_forward(amt)
        self.update_knowledge(amt, success_of_probing)
        self.learn_n_bits(oracle,n-1)
    


# Uncertainty Network



In [9]:


class UncertaintyNetwork(ChannelGraph):  
    """
    The UncertaintayNetwork is the main data structure to store our belief about the 
    Liquidity in the channels of the ChannelGraph.
    
    Most of its functionality comes from the UncertaintyChannel. Most notably the ability
    to assign a linearized integer uncertainty unit cost to its channels and do this even
    piecewise.
    
    Paths cannot be probed against the UncertaintyNetwork as it lacks an Oracle
    """
    def __init__(self,channel_graph:ChannelGraph, base_threshold : int = DEFAULT_BASE_THRESHOLD):
        self._channel_graph = nx.MultiDiGraph()
        for src,dest, keys, channel in channel_graph.network.edges(data="channel", keys=True):
            oracle_channel = UncertaintyChannel(channel)
            if channel.base_fee <= base_threshold:
                self._channel_graph.add_edge(oracle_channel.src, 
                                             oracle_channel.dest, 
                                             key=oracle_channel.short_channel_id, 
                                             channel=oracle_channel)

    @property
    def network(self):
        return self._channel_graph
    
    def entropy(self):
        """
        computes to total uncertainty in the network summing the entropy of all channels
        """
        return sum(channel.entropy() for src,dest, channel in self.network.edges(data = "channel"))
    
    
    def get_features_of_candidate_path(self,path: List[UncertaintyChannel], amt: int) -> (float, float):
        """
        returns the routing fees and probability of a candidate path
        """
        probability = 1
        routing_fees = 0
        for channel in path:
            routing_fees += channel.routing_cost_msat(amt)
            probability *= channel.success_probability(amt)
        return routing_fees, probability

    def allocate_amount_on_path(self,path: List[UncertaintyChannel], amt: int):
        """
        allocates `amt` to all channels of the path of `UncertaintyChannels`
        """
        for channel in path:
            channel.allocate_amount(amt)        

    def reset_uncertainty_network(self):
        """
        resets our belief about the liquidity & inflight information of all channels on the UncertaintyNetwork
        """
        for src,dest, channel in self.network.edges(data = "channel"):
            channel.forget_information()
    
    def activate_network_wide_uncertainty_reduction(self,n,oracle:OracleLightningNetwork):
        """
        With the help of an `OracleLightningNetwork` probes all chennels `n` times to reduce uncertainty.
        
        While one can do this on mainnet by probing we can do this very quickly in simulation
        at virtually no cost. Thus this API call needs to be taken with caution when using a different
        oracle. 
        """
        for src,dest, channel in self.network.edges(data = "channel"):
            channel.learn_n_bits(oracle,n)
            
    #FIXME: refactor to new code base. The following call will break!
    def activate_foaf_uncertainty_reduction(self,src,dest):
        ego_netwok=set()
        foaf_network = set()

        out_set = set()
        edges = self.__channel_graph.out_edges(self.__node_key_to_id[src])
        for edge in edges:
            ego_netwok.add("{}x{}".format(edge[0],edge[1]))
            out_set.add(edge[1])
        
        for node in out_set:
            edges = self.__channel_graph.out_edges(node)
            for edge in edges:
                foaf_network.add("{}x{}".format(edge[0],edge[1]))

        in_set = set()
        edges = self.__channel_graph.in_edges(self.__node_key_to_id[dest])
        for edge in edges:
            ego_netwok.add("{}x{}".format(edge[0],edge[1]))
            in_set.add(edge[0])
        
        for node in in_set:
            edges = self.__channel_graph.out_edges(node)
            for edge in edges:
                foaf_network.add("{}x{}".format(edge[0],edge[1]))
                
                
        #print(len(ego_netwok))
        for k,arc in self.__arcs.items():
            #print(k)
            vals = k.split("x")
            key = "{}x{}".format(vals[0],vals[1])
            if key in ego_netwok:
                l = arc.get_actual_liquidity()
                arc.update_knowledge(l-1)
                arc.update_knowledge(l+1)
                #print(key,arc.entropy())
            
            if key in foaf_network:
                arc.learn_n_bits(2)
                l = arc.get_actual_liquidity()
                arc.update_knowledge(l-1)
                arc.update_knowledge(l+1)
                #print(key, arc.entropy())
        print("channels with full knowlege: ", len(ego_netwok))
        print("channels with 2 Bits of less entropy: ", len(foaf_network))

# Payment Session

Payments are conducted within a payment session. The payment session needs to be given an instance from the` UncertaintyNetwork` and an `Oracle` against which it will `send_onions`. In this notebook the `Oracel` is just a simulated distribution of the Liquidity in the network. For experiments we could distribute the liquidity differently or we could use the mainnet network or a crawl as the oracle. In particular the `UncertaintyNetwork` may contain prior belief about the uncertainty of the liquidity in remote channels.

The main API call in a payment session is `pickhardt_pay` which is a `pay`-implementation of our method. Internally `pickhardt_pay` feeds the min cost solver with `pieceweise linearized integer unit costs` for all channels of the uncertainty network that are not to be pruned. The pruning is currently pretty arbitrary and exists to show that another order of magnitude in compuational time seems possible without loosing much optimality but shall be chosen better for future extensability. (I assume with proper feature engineering one may prune based on unit costs).
The min cost flow solver produces a flow that is disected into candidate path. instead of just invoking `send_onion` and makting payment attempts the `pickhardt_pay` loop here collects our belief about success probabilities and fees so that we can investigate the results of the simulation. These results are also depicted as part of the API call. 

In [34]:
class PaymentSession():
    """
    A PaymentSesssion is used to create the min cost flow problem from the UncertaintyNetwork
    
    This happens by adding several parallel arcs coming from the piece wise linearization of the
    UncertaintyChannel to the min_cost_flow object. 
    
    The main API call ist `pickhardt_pay` which invokes a sequential loop to conduct trial and error
    attmpts. The loop could easily send out all onions concurrently but this does not make sense 
    against the simulated OracleLightningNetwork. 
    """
        
    def __init__(self, 
                 oracle : OracleLightningNetwork, 
                 uncertainty_network: UncertaintyNetwork,
                 prune_network : bool =True):
        self._oracle = oracle
        self._uncertainty_network = uncertainty_network
        self._prune_network=prune_network
        self._prepare_integer_indices_for_nodes()
        
    def _prepare_integer_indices_for_nodes(self):
        """
        necessary for the OR-lib by google and the min cost flow solver
        

        let's initialize the look up tables for node_ids to integers from [0,...,#number of nodes]
        this is necessary because of the API of the Google Operations Research min cost flow solver
        """
        self._mcf_id = {}
        self._node_key = {}
        for k, node_id in enumerate(self._uncertainty_network.network.nodes()):
            self._mcf_id[node_id]=k
            self._node_key[k]=node_id

    
    def _prepare_mcf_solver(self, src, dest,amt :int=1, mu : int =100_000_000, base_fee : int =DEFAULT_BASE_THRESHOLD):
        """
        computes the uncertainty network given our prior belief and prepares the min cost flow solver

        This function can define a value for \mu to control how heavily we combine the uncertainty cost and fees
        Also the function supports only taking channels into account that don't charge a base_fee higher or equal to `base`

        returns the instantiated min_cost_flow object from the google OR-lib that contains the piecewise linearized problem
        """
        self._min_cost_flow = pywrapgraph.SimpleMinCostFlow()
        self._arc_to_channel = {}
        
        for s, d, channel in self._uncertainty_network.network.edges(data="channel"):
            #ignore channels with too large base fee
            if channel.base_fee > base_fee:
                continue
            #FIXME: Remove Magic Number for pruning
            # Prune channels away thay have too low success probability! This is a huge runtime boost
            # However the pruning would be much better to work on quantiles of normalized cost
            # So as soon as we have better Scaling, Centralization and feature engineering we can 
            # probably have a more focused pruning
            if self._prune_network and channel.success_probability(250_000)<0.9:
                continue
            cnt = 0
            for capacity, cost in channel.get_piecewise_linearized_costs(mu=mu): #QUANTIZATION):
                index = self._min_cost_flow.AddArcWithCapacityAndUnitCost(self._mcf_id[s], 
                                                            self._mcf_id[d], 
                                                            capacity,
                                                            cost)
                self._arc_to_channel[index]=(s,d,channel,0)
                if self._prune_network and cnt>1:
                    break
                cnt+=1

        # Add node supply to 0 for all nodes
        for i in self._uncertainty_network.network.nodes():
            self._min_cost_flow.SetNodeSupply(self._mcf_id[i], 0)

        #add amount to sending node
        self._min_cost_flow.SetNodeSupply(self._mcf_id[src],int(amt))#/QUANTIZATION))

        #add -amount to recipient nods
        self._min_cost_flow.SetNodeSupply(self._mcf_id[dest],-int(amt))#/QUANTIZATION))

    def _next_hop(self,path):
        """
        generator to iterate through edges indext by node id of paths

        The path is a list of node ids. Each call returns a tuple src, dest of an edge in the path    
        """
        for i in range(1,len(path)):
            src = path[i-1]
            dest = path[i]
            yield (src,dest)
        
    def _make_channel_path(self,G :nx.MultiDiGraph, path: List[str]):
        """
        network x returns a path as a list of node_ids. However we need a list of `UncertaintyChannels`
        Since the graph has parallel edges it is quite some work to get the actual channels that the 
        min cost flow solver produced
        """
        channel_path = []
        bottleneck = 2**63
        for src,dest in self._next_hop(path):
            w = 2**63
            c = None
            flow = 0
            for sid in G[src][dest].keys():
                if G[src][dest][sid]["weight"]< w:
                    w = G[src][dest][sid]["weight"]
                    c = G[src][dest][sid]["channel"]
                    flow = G[src][dest][sid]["flow"]
            channel_path.append(c)
            
            if flow < bottleneck:
                bottleneck = flow
                
        return channel_path,bottleneck

    def _disect_flow_to_paths(self,s,d):
        """
        A standard algorithm to disect a flow into several paths.

        FIXME: Note that this disection while accurate is probably not optimal in practise. 
        As noted in our Probabilistic payment delivery paper the payment process is a bernoulli trial 
        and I assume it makes sense to disect the flow into paths of similar likelihood to make most
        progress but this is a mere conjecture at this point. I expect quite a bit of research will be
        necessary to resolve this issue.
        """
        total_flow = {}

        #first collect all linearized edges which are assigned a non zero flow put them into a networkx graph
        G = nx.MultiDiGraph()
        for i in range(self._min_cost_flow.NumArcs()):
            flow = self._min_cost_flow.Flow(i)#*QUANTIZATION
            if flow == 0:
                continue

            
            src, dest, channel,_ = self._arc_to_channel[i]
            if G.has_edge(src,dest):
                if channel.short_channel_id in G[src][dest]:
                    G[src][dest][channel.short_channel_id]["flow"]+=flow
            else:
                #FIXME: cost is not reflecting exactly the piecewise linearization
                #Probably not such a big issue as we just disect flow
                G.add_edge(src,dest,key=channel.short_channel_id,flow=flow,channel=channel, weight=channel.combined_linearized_unit_cost())
        used_flow = 1
        channel_paths = []
        
        #allocate flow to shortest / cheapest paths from src to dest as long as this is possible
        #decrease flow along those edges. This is a standard mechanism to disect a flow int paths
        while used_flow>0:
            path = None
            try: 
                path = nx.shortest_path(G,s,d)
            except: 
                break
            channel_path,used_flow = self._make_channel_path(G,path) 
            channel_paths.append((channel_path,used_flow))
            
            #reduce the flow from the selected path
            for pos,hop in enumerate(self._next_hop(path)):
                src, dest = hop
                channel = channel_path[pos]
                G[src][dest][channel.short_channel_id]["flow"]-=used_flow
                if G[src][dest][channel.short_channel_id]["flow"]==0:
                    G.remove_edge(src,dest,key=channel.short_channel_id)
        return channel_paths
        
    def _generate_candidate_paths(self,src, dest, amt,mu: int =100_000_000,base : int =DEFAULT_BASE_THRESHOLD):
        """
        computes the optimal payment split to deliver `amt` from `src` to `dest` and updates our belief about the liquidity

        This is one step within the payment loop.

        Retuns the residual amount of the `amt` that could ne be delivered and the paid fees
        (on a per channel base not including fees for downstream fees) for the delivered amount

        the function also prints some results an statistics about the paths of the flow to stdout.
        """
        
        #First we prepare the min cost flow by getting arcs from the uncertainty network
        self._prepare_mcf_solver(src,dest,amt,mu,base)   

        start = time.time()
        #print("solving mcf...")
        status = self._min_cost_flow.Solve()
        
        if status != self._min_cost_flow.OPTIMAL:
            print('There was an issue with the min cost flow input.')
            print(f'Status: {status}')
            exit(1)

        paths = self._disect_flow_to_paths(src,dest)
        end = time.time()
        return paths, end-start
    
    
    def _estimate_payment_statistics(self,paths):
        """
        estimates the success probability of paths and computes fees (without paying downstream fees)
        
        @returns the statistics in the `payments` dictionary
        """
        #FIXME: Decide if an `Payments` or `Attempt` class shall be used
        payments = {}
        #compute fees and probabilities of candidate paths for evaluation
        for i, onion in enumerate(paths):
            path, amount = onion
            fee, probability = self._uncertainty_network.get_features_of_candidate_path(path,amount)
            payments[i] = {"routing_fee": fee, "probability": probability, "path": path, "amount": amount}
            
            #to correctly compute conditional probabilities of non disjoint paths in the same set of paths
            self._uncertainty_network.allocate_amount_on_path(path,amount)

        #remove allocated amounts for all planned onions before doing actual attempts 
        for key, attempt in payments.items():
            self._uncertainty_network.allocate_amount_on_path(attempt["path"],-attempt["amount"])
    
        return payments
        
    def _attempt_payments(self, payments):
        """
        we attempt all planned payments and test the success against the oracle in particular this
        method changes - depending on the outcome of each payment - our belief about the uncertainty
        in the UncertaintyNetwork
        """
        #test actual payment attempts
        for key, attempt in payments.items():
            success, erring_channel = self._oracle.send_onion(attempt["path"],attempt["amount"])
            payments[key]["success"] = success
            payments[key]["erring_channel"] = erring_channel
            if success:
                self._uncertainty_network.allocate_amount_on_path(attempt["path"],attempt["amount"])    
    
    def _evaluate_attempts(self,payments):
        """
        helper function to collect statistics about attempts and print them
        
        returns the `residual` amount that could not have been delivered and some statistics
        """
        total_fees = 0
        paid_fees = 0
        residual_amt = 0
        number_failed_paths = 0
        expected_sats_to_deliver = 0
        amt = 0
        print("\nStatistics about {} candidate onions:\n".format(len(payments)))
        
        has_failed_attempt = False
        print("successful attempts:")
        print("--------------------")
        for attempt in payments.values():
            success = attempt["success"]
            if success == False:
                has_failed_attempt = True
                continue
            fee = attempt["routing_fee"] / 1000.
            probability = attempt["probability"]
            path = attempt["path"]
            amount = attempt["amount"]
            amt += amount
            total_fees += fee
            expected_sats_to_deliver += probability * amount
            print(" p = {:6.2f}% amt: {:9} sats  hops: {} ppm: {:5}".format(probability*100, amount, len(path),int(fee*1000_000/amount)))
            paid_fees += fee
        
        if has_failed_attempt:
            print("\nfailed attempts:")
            print("----------------")
            for attempt in payments.values():
                success = attempt["success"]
                if success:
                    continue
                fee = attempt["routing_fee"] / 1000.
                probability = attempt["probability"]
                path = attempt["path"]
                amount = attempt["amount"]
                amt += amount
                total_fees += fee
                expected_sats_to_deliver += probability * amount
                print(" p = {:6.2f}% amt: {:9} sats  hops: {} ppm: {:5} ".format(probability*100, amount, len(path),int(fee*1000_000/amount)))
                number_failed_paths += 1
                residual_amt += amount

                
        print("\nAttempt Summary:")
        print("=================")
        print("\nTried to deliver {:10} sats".format(amt))
        fraction = expected_sats_to_deliver*100./amt
        print("expected to deliver {:10} sats \t({:4.2f}%)".format(int(expected_sats_to_deliver),fraction))
        fraction = (amt-residual_amt)*100./(amt)
        print("actually deliverd {:10} sats \t({:4.2f}%)".format(amt-residual_amt,fraction))
        print("deviation: {:4.2f}".format( (amt-residual_amt)/(expected_sats_to_deliver+1)))
        print("planned_fee: {:8.3f} sat".format(total_fees))
        print("paid fees: {:8.3f} sat".format(paid_fees))
        return residual_amt, paid_fees, len(payments), number_failed_paths

    
    def forget_information(self):
        """
        forgets all the information in the UncertaintyNetwork that is a member of the PaymentSession
        """
        self._uncertainty_network.reset_uncertainty_network()
        
    def activate_network_wide_uncertainty_reduction(self,n):
        """
        Pipes API call to the UncertaintyNetwork
        """
        self._uncertainty_network.activate_network_wide_uncertainty_reduction(n,self._oracle)

    
    def pickhardt_pay(self,src,dest,amt,mu=1,base=0):
        """
        conduct one experiment! might need to call oracle.reset_uncertainty_network() first
        I could not put it here as some experiments require sharing of liqudity information

        """    
        entropy_start=self._uncertainty_network.entropy()
        start = time.time()
        full_amt = amt
        cnt = 0
        total_fees = 0
        number_number_of_onions = 0
        total_number_failed_paths = 0
        
        #This is the main payment loop. It is currently blocking and synchronous but may be 
        #implemented in a concurrent way. Also we stop after 10 rounds which is pretty arbitrary
        #a better stop criteria would be if we compute infeasable flows or if the probabilities 
        #are to low or residual amounts decrease to slowly
        while amt > 0 and cnt < 10:
            print("Round number: ", cnt+1)
            print("Try to deliver", amt, "satoshi:")
            
            #transfer to a min cost flow problem and rund the solver
            paths,runtime = self._generate_candidate_paths(src,dest,amt,mu,base)
            
            #compute some statistics about candidate paths
            payments = self._estimate_payment_statistics(paths)
            
            #matke attempts and update our information about the UncertaintyNetwork
            self._attempt_payments(payments)
            
            #run some simple statistics and depict them
            amt, paid_fees, num_paths, number_failed_paths = self._evaluate_attempts(payments)
            print("Runtime of flow computation: {:4.2f} sec ".format(runtime))
            print("\n================================================================\n")

            number_number_of_onions += num_paths
            total_number_failed_paths+=number_failed_paths
            total_fees += paid_fees
            cnt+=1
        end = time.time()
        entropy_end = self._uncertainty_network.entropy()
        print("SUMMARY:")
        print("========")
        print("Rounds of mcf-computations: ", cnt)
        print("Number of onions sent: ", number_number_of_onions)
        print("Number of failed onions: ", total_number_failed_paths)
        print("Failure rate: {:4.2f}% ".format(total_number_failed_paths*100./number_number_of_onions))
        print("total runtime (including inefficient memory managment): {:4.3f} sec".format(end-start))
        print("Learnt entropy: {:5.2f} bits".format(entropy_start-entropy_end))
        print("Fees for successfull delivery: {:8.3f} sat --> {} ppm".format(total_fees,int(total_fees*1000*1000/full_amt)))
        print("used mu:", mu)

# Do experiments and apply the API

We can now use the `PaymentSession` to conduct several experiments by calling `pickhardt_pay`.
We may of course exchange the Oracle or modify our belief by manipulating the UncertaintyNetwork

In [29]:
#Rene Pickhardt's public node_key
RENE = "03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df"
#Carsten Otto's public node key
C_OTTO = "027ce055380348d7812d2ae7745701c9f93e70c1adeb2657f053f91df4f2843c71"
N = 5

channel_graph = ChannelGraph("listchannels20220412.json")
oracle_lightning_network = OracleLightningNetwork(channel_graph)
uncertainty_network = UncertaintyNetwork(channel_graph)

In [30]:
max_possible =oracle_lightning_network.theoretical_maximum_payable_amount(RENE,C_OTTO,1000)
print(max_possible, "sats would be possible on this oracle to deliver if including 1 sat basefee channels")
max_possible =oracle_lightning_network.theoretical_maximum_payable_amount(RENE,C_OTTO,0)
print(max_possible, "sats possible on this oracle on the zeroBaseFeeGraph")

133317516 sats would be possible on this oracle to deliver if including 1 sat basefee channels
81606959 sats possible on this oracle on the zeroBaseFeeGraph


## make highly reliable payment on the zeroBaseFee Network

In [38]:
AMT = int(max_possible/2)
payment_session = PaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=False)
payment_session.forget_information()
#payment_session.activate_network_wide_uncertainty_reduction(1)

payment_session.pickhardt_pay(RENE,C_OTTO, AMT,mu=0,base=0)

Round number:  1
Try to deliver 40803479 satoshi:

Statistics about 12 candidate onions:

successful attempts:
--------------------
 p =  52.92% amt:   2800000 sats  hops: 3 ppm:  1897
 p =  68.30% amt:   2020000 sats  hops: 3 ppm:  1974
 p =  69.25% amt:    630528 sats  hops: 4 ppm:  3717
 p =  59.67% amt:    200000 sats  hops: 4 ppm: 21787
 p =  70.46% amt:    858430 sats  hops: 4 ppm:  2186
 p =  48.33% amt:   3355443 sats  hops: 4 ppm:  2496
 p =  48.11% amt:    969011 sats  hops: 5 ppm:  2057
 p =  43.50% amt:   3000000 sats  hops: 5 ppm:  2580
 p =  20.97% amt:   5046329 sats  hops: 5 ppm:  2551

failed attempts:
----------------
 p =   3.89% amt:  15412696 sats  hops: 3 ppm:  2036 
 p =  70.31% amt:   1000000 sats  hops: 4 ppm:  2060 
 p =  17.01% amt:   5369472 sats  hops: 6 ppm:  5317 

Attempt Summary:

Tried to deliver   40661909 sats
expected to deliver   10689556 sats 	(26.29%)
actually deliverd   18879741 sats 	(46.43%)
deviation: 1.77
planned_fee: 110879.668 sat
paid fee

## make very cheap payment on the zeroBaseFee Network without Pruning

currently the pruning strategy is mainly implemented on a reliability estimate thus pruning when optimizing for fees does not make sense

In [39]:
payment_session = PaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=False)
payment_session.forget_information()

payment_session.pickhardt_pay(RENE,C_OTTO, AMT,mu=1000, base =0)

Round number:  1
Try to deliver 40803479 satoshi:

Statistics about 33 candidate onions:

successful attempts:
--------------------
 p =  57.70% amt:    384000 sats  hops: 2 ppm:  1342
 p =  72.45% amt:    194200 sats  hops: 3 ppm:  1362
 p =  63.79% amt:    400000 sats  hops: 3 ppm:  1347
 p =   5.33% amt:   1000000 sats  hops: 4 ppm:  1368
 p =   0.00% amt:   1188865 sats  hops: 5 ppm:  1352
 p =  20.54% amt:   1000000 sats  hops: 5 ppm:  1357
 p =   4.16% amt:   3200000 sats  hops: 5 ppm:  1362
 p =   0.27% amt:    605279 sats  hops: 6 ppm:  1361
 p =  28.63% amt:    400000 sats  hops: 6 ppm:  1339
 p =   1.35% amt:    224144 sats  hops: 6 ppm:  1364
 p =  16.27% amt:    400000 sats  hops: 6 ppm:  1351
 p =  42.28% amt:    200000 sats  hops: 8 ppm:  1345
 p =  13.09% amt:     20000 sats  hops: 11 ppm:  1343

failed attempts:
----------------
 p =  31.20% amt:    660000 sats  hops: 2 ppm:  1339 
 p =   0.00% amt:   5000000 sats  hops: 2 ppm:  1360 
 p =  72.00% amt:    200000 sats  h


Statistics about 43 candidate onions:

successful attempts:
--------------------
 p =  79.76% amt:     27648 sats  hops: 2 ppm:  1341
 p =  75.73% amt:     36864 sats  hops: 3 ppm:  1336
 p =  69.26% amt:    156553 sats  hops: 4 ppm:  1389
 p =  66.20% amt:    229666 sats  hops: 4 ppm:  1365
 p =  76.74% amt:     35388 sats  hops: 4 ppm:  1408
 p =  90.39% amt:     43200 sats  hops: 4 ppm:  1343
 p =  88.11% amt:     22127 sats  hops: 4 ppm:  1363
 p =  44.52% amt:    200000 sats  hops: 5 ppm:  1395
 p =  75.78% amt:    136000 sats  hops: 5 ppm:  1395
 p =  90.28% amt:     80000 sats  hops: 5 ppm:  1385
 p =  74.40% amt:      2841 sats  hops: 6 ppm:  1367
 p =  72.37% amt:     43200 sats  hops: 6 ppm:  1342
 p =  72.69% amt:     56250 sats  hops: 6 ppm:  1364
 p =  60.91% amt:    136000 sats  hops: 7 ppm:  1396
 p =  59.36% amt:    152866 sats  hops: 7 ppm:  1386
 p =   2.27% amt:    134091 sats  hops: 7 ppm:  1342
 p =  28.40% amt:    225574 sats  hops: 7 ppm:  1396
 p =  72.42% amt:


Statistics about 7 candidate onions:

successful attempts:
--------------------
 p =  84.60% amt:     98566 sats  hops: 7 ppm:  1392
 p =  92.80% amt:     64000 sats  hops: 7 ppm:  1390
 p =  79.85% amt:     29434 sats  hops: 7 ppm:  1396
 p =  53.13% amt:    131472 sats  hops: 10 ppm:  1398
 p =  67.23% amt:    106000 sats  hops: 11 ppm:  1347
 p =  46.01% amt:     64000 sats  hops: 12 ppm:  1347

failed attempts:
----------------
 p =   0.00% amt:   4869520 sats  hops: 2 ppm:  1359 

Attempt Summary:

Tried to deliver    5362992 sats
expected to deliver     336841 sats 	(6.28%)
actually deliverd     493472 sats 	(9.20%)
deviation: 1.46
planned_fee: 7302.838 sat
paid fees:  680.292 sat
Runtime of flow computation: 0.69 sec 


SUMMARY:
Rounds of mcf-computations:  10
Number of onions sent:  276
Number of failed onions:  128
Failure rate: 46.38% 
total runtime (including inefficient memory managment): 16.823 sec
Learnt entropy: 342.98 bits
Fees for successfull delivery: 47160.204 sat -

## Make highly reliable payment with the pruned zeroBaseFee network

In [40]:
payment_session = PaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=True)
payment_session.forget_information()
#payment_session.activate_network_wide_uncertainty_reduction(1)

payment_session.pickhardt_pay(RENE,C_OTTO, AMT,mu=0,base=0)

Round number:  1
Try to deliver 40803479 satoshi:

Statistics about 16 candidate onions:

successful attempts:
--------------------
 p =  75.20% amt:    600000 sats  hops: 2 ppm:  2572
 p =  52.92% amt:   2800000 sats  hops: 3 ppm:  1897
 p =  68.30% amt:   2020000 sats  hops: 3 ppm:  1974
 p =  69.25% amt:    630528 sats  hops: 4 ppm:  3717
 p =  65.76% amt:   1000000 sats  hops: 4 ppm:  2187
 p =  47.42% amt:   3355443 sats  hops: 4 ppm:  2496
 p =  96.83% amt:     61199 sats  hops: 4 ppm:  2481
 p =  71.04% amt:    640000 sats  hops: 4 ppm:  2564
 p =  59.67% amt:    200000 sats  hops: 4 ppm: 21787
 p =  43.95% amt:   3000000 sats  hops: 5 ppm:  2580
 p =  20.38% amt:   5046329 sats  hops: 5 ppm:  2551

failed attempts:
----------------
 p =  25.52% amt:  10066329 sats  hops: 3 ppm:  2036 
 p =  48.04% amt:   2000000 sats  hops: 4 ppm:  2060 
 p =  66.75% amt:    882941 sats  hops: 4 ppm: 21456 
 p =  23.08% amt:   3131238 sats  hops: 5 ppm:  2057 
 p =  15.80% amt:   5369472 sats  

## Make very cheap payment on the pruned zeroBaseFee Network
note that pruning still removes unreliable channels thus this method already provides a tradeoff between routing fees and realiability 

In [41]:
payment_session = PaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=True)
payment_session.forget_information()
payment_session.pickhardt_pay(RENE,C_OTTO, AMT,mu=1000,base=0)

Round number:  1
Try to deliver 40803479 satoshi:

Statistics about 31 candidate onions:

successful attempts:
--------------------
 p =  16.00% amt:   3000000 sats  hops: 2 ppm:  1360
 p =  53.36% amt:    500000 sats  hops: 4 ppm:  1694
 p =  23.04% amt:   1000000 sats  hops: 4 ppm:  1368
 p =  25.75% amt:   1000000 sats  hops: 5 ppm:  1357
 p =  11.39% amt:   1862000 sats  hops: 5 ppm:  1362
 p =  14.20% amt:    323770 sats  hops: 5 ppm:  1349
 p =  41.83% amt:    948823 sats  hops: 6 ppm:  1703
 p =  25.28% amt:    800000 sats  hops: 6 ppm:  1351
 p =  25.61% amt:    262911 sats  hops: 6 ppm:  1496
 p =  20.22% amt:   1075089 sats  hops: 7 ppm:  1696
 p =   5.44% amt:    124911 sats  hops: 8 ppm:  1696
 p =  22.46% amt:    351177 sats  hops: 8 ppm:  1701
 p =   3.18% amt:      4329 sats  hops: 11 ppm:  1358

failed attempts:
----------------
 p =  25.37% amt:   2737089 sats  hops: 3 ppm:  1487 
 p =  26.97% amt:   3000000 sats  hops: 3 ppm:  1352 
 p =  14.13% amt:   1920000 sats  h


Statistics about 6 candidate onions:

successful attempts:
--------------------
 p =  96.79% amt:     59535 sats  hops: 6 ppm:  1734
 p =  88.68% amt:    300878 sats  hops: 7 ppm:  1731
 p =  87.79% amt:     59535 sats  hops: 12 ppm:  1713
 p =  79.95% amt:     66401 sats  hops: 12 ppm:  1718

failed attempts:
----------------
 p =  69.88% amt:    452465 sats  hops: 8 ppm:  1727 
 p =  18.98% amt:    474064 sats  hops: 15 ppm:  1506 

Attempt Summary:

Tried to deliver    1412878 sats
expected to deliver     835933 sats 	(59.17%)
actually deliverd     486349 sats 	(34.42%)
deviation: 0.58
planned_fee: 2336.861 sat
paid fees:  840.593 sat
Runtime of flow computation: 0.13 sec 


Round number:  8
Try to deliver 926529 satoshi:

Statistics about 3 candidate onions:

successful attempts:
--------------------
 p =  70.49% amt:    420981 sats  hops: 7 ppm:  1726
 p =  68.96% amt:     53083 sats  hops: 10 ppm:  1519
 p =  34.73% amt:    452465 sats  hops: 15 ppm:  1508

Attempt Summary:

Tri

## Test the effect of network wide probing and using that information for cheap payments

In [44]:
payment_session = PaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=True)
payment_session.forget_information()
payment_session.activate_network_wide_uncertainty_reduction(1)

payment_session.pickhardt_pay(RENE,C_OTTO, AMT,mu=1000,base=0)

Round number:  1
Try to deliver 40803479 satoshi:

Statistics about 61 candidate onions:

successful attempts:
--------------------
 p =  25.00% amt:   3750000 sats  hops: 2 ppm:  1360
 p = 100.00% amt:    950000 sats  hops: 3 ppm:  1687
 p =  73.38% amt:    869527 sats  hops: 3 ppm:  1365
 p =  55.07% amt:   1000000 sats  hops: 3 ppm:  1488
 p =  60.51% amt:    500000 sats  hops: 3 ppm:  1438
 p =  43.28% amt:   1417904 sats  hops: 3 ppm:  1351
 p =  64.00% amt:    500000 sats  hops: 4 ppm:  1357
 p = 100.00% amt:   2500000 sats  hops: 4 ppm:  1368
 p =  40.00% amt:    188642 sats  hops: 5 ppm:  1367
 p =  75.00% amt:   3125000 sats  hops: 5 ppm:  1357
 p = 100.00% amt:    800000 sats  hops: 5 ppm:  1359
 p =  80.00% amt:    500000 sats  hops: 5 ppm:  1354
 p =  75.18% amt:    332363 sats  hops: 5 ppm:  1351
 p =  25.03% amt:   1102233 sats  hops: 5 ppm:  1366
 p =  88.77% amt:    561358 sats  hops: 6 ppm:  1368
 p =  68.33% amt:    250000 sats  hops: 6 ppm:  1445
 p =  64.84% amt:   

## Make a payment that optimizes for both reliability and fees with no probing

In [46]:
payment_session = PaymentSession(oracle_lightning_network, 
                                 uncertainty_network,
                                 prune_network=True)
payment_session.forget_information()
payment_session.pickhardt_pay(RENE,C_OTTO, AMT,mu=1,base=0)

Round number:  1
Try to deliver 40803479 satoshi:

Statistics about 18 candidate onions:

successful attempts:
--------------------
 p =  74.60% amt:    600000 sats  hops: 3 ppm:  2092
 p =  52.92% amt:   2800000 sats  hops: 3 ppm:  1897
 p =  96.81% amt:     61199 sats  hops: 3 ppm:  1681
 p =  77.71% amt:   1371238 sats  hops: 3 ppm:  2379
 p =  45.91% amt:   3355443 sats  hops: 3 ppm:  1750
 p =  61.96% amt:   2020000 sats  hops: 3 ppm:  1974
 p =  67.71% amt:    640000 sats  hops: 3 ppm:  1564
 p =  36.50% amt:   3200000 sats  hops: 4 ppm:  2603
 p =  27.71% amt:   1800000 sats  hops: 4 ppm:  2118
 p =  25.68% amt:   1200000 sats  hops: 4 ppm:  1844
 p =  68.50% amt:   1000000 sats  hops: 4 ppm:  2187
 p =  30.61% amt:   4000000 sats  hops: 4 ppm:  1552
 p =  30.60% amt:   1628762 sats  hops: 5 ppm:  2579
 p =  22.81% amt:   1046329 sats  hops: 5 ppm:  1709

failed attempts:
----------------
 p =  25.52% amt:  10066329 sats  hops: 3 ppm:  2036 
 p =  44.78% amt:   2000000 sats  hop


# Backup code












## Probably depricated functions and todos for refactoring from old code base

In [ ]:
    def __init__(self,filename:str):
        """
        It starts by importing the channel_graph from c-lightning listchannels command and then 
        """
        self._network = nx.MultiDiGraph()
        channels = self.get_channel_json(filename)
        for channel in channels:
            oracle_channel = OracleChannel(channel)
            self._channel_graph.add_edge(oracle_channel.src, 
                                          oracle_channel.dest, 
                                          key=oracle_channel.short_channel_id, 
                                          channel=oracle_channel)
            
            
    def get_all_parallel_channels(self, src: str, dest: str):
        """
        FIXME: depricate? It's
        """
        if self.network.has_edge(src,dest):
            return {short_channel_id : self.network[src][dest][short_channel_id]["channel"] 
                    for short_channel_id in self.network[src][dest].keys()}

        
    def __init__(self,filename:str, base_threshold : int = DEFAULT_BASE_THRESHOLD):
        """
        It starts by importing the channel_graph from c-lightning listchannels command and then 
        """
        self._channel_graph = nx.MultiDiGraph()
        channels = self.get_channel_json(filename)
        for channel in channels:
            uncertainty_channel = UncertaintyChannel(channel)
            if channel.base_fee <= base_threshold:
                self._channel_graph.add_edge(oracle_channel.src, 
                                             oracle_channel.dest, 
                                             key=oracle_channel.short_channel_id, 
                                             channel=oracle_channel)

                
        """
        #allocate amounts for probability computation
        for i, onion in enumerate(paths):
            path, amount = onion
            fee, probability = uncertainty_network.allocate_path(path,amount)
            payments[i] = {"routing_fee": fee, "probability": probability, "path": path, "amount": amount}

        #remove allocated amounts for all onions ---#failed onions
        for key, attempt in payments.items():
            #if attempt["success"] == False:
            self._uncertainty_network.allocate_path(attempt["path"],-attempt["amount"])

        #test actual payment attempts
        for key, attempt in payments.items():
            payments[key]["success"] = uncertainty_network.probe_path(attempt["path"],attempt["amount"])
        """

    def __init__(self,filename:str):
        channel_graph = ChannelGraph(filename)
        oracle = OracleLightningNetwork(self._channel_graph)
        uncertainty_network = UncertaintyNetwork(self._channel_graph)
        self.__init__(channel_graph,oracle, uncertainty_network)

        
    def __init__(self,cln_jsn : dict,actual_liquidity : int = None):
        super().__init__(cln_jsn)
        self._actual_liquidity=actual_liquidity
        if actual_liquidity is None or actual_liquidity >= self.capacity or actual_liquidity < 0:
            self._actual_liquidity = random.randint(0,self.capacity)


In [ ]:


        """
    #FIXME: Needs this method in ChannelGraph
    def get_next_channel(self,src:str,dest:str):
        
        if self.network.has_edge(src,dest):
            for k,channel in self.network[src][dest].items():
                print(k)
                yield channel["channel"]
        """


for channel in channel_graph.get_next_channel(src,dest):
    print(channel.short_channel_id)
    print(channel)
    
    print(channel.entropy())
    print(channel.success_probability(20000))
    print(channel.uncertainty_cost(20000))
    print()
    channel.min_liquidity=10000
    print(channel.success_probability(20000))
    print(channel.uncertainty_cost(20000))
    print()
    channel.max_liquidity=30000
    print(channel.success_probability(20000))
    print(channel.uncertainty_cost(20000))
    print("ppm: {} base: {}".format(channel.ppm, channel.base_fee))
    print(channel.routing_cost_msat(20000))
    
    print(channel.success_probability(20000))
    print(channel.combined_linearized_unit_cost())
    print(channel.linearized_integer_uncertainty_unit_cost())
    
    print("ppm: {} base: {}".format(channel.ppm, channel.base_fee))
    print(channel.routing_cost_msat(20000))
    channel.forget_information()
    channel.min_liquidity=120000
    
    MU = UncertaintyChannel.TOTAL_NUMBER_OF_SATS / 21_000_000
    for p in channel.get_piecewise_linearized_costs(number_of_pieces=5,quantization=100000,mu=MU*1):
        print(p)
        
    for p in channel.get_piecewise_linearized_costs(number_of_pieces=5,quantization=100000,mu=MU*10):
        print(p)
    
    for p in channel.get_piecewise_linearized_costs(number_of_pieces=5,quantization=100000,mu=MU*100):
        print(p)
        print(log(p[1])+ log(UncertaintyChannel.TOTAL_NUMBER_OF_SATS/100_000/100))
    break

In [31]:
#via: https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
#and: https://developers.google.com/optimization/install
import sys
!{sys.executable} -m pip install --upgrade --user ortools
#!conda install --yes --prefix {sys.prefix} -c conda-forge ortools-python